In [9]:
from splinter import Browser
from bs4 import BeautifulSoup
import datetime
import os
import json
import time
import re

# Sets the root url
root_url = 'https://www.glassdoor.com'

# Sets the time period for the search
time_period = "Last Month"

# Set path variables
log_path = '/Users/grothjd/DS_bootcamp/jobsearch_dashboard/log/'
corpus_path = '/Users/grothjd/DS_bootcamp/jobsearch_dashboard/corpus2/'

meta_message = 'Searching for all job posting from the last Month'

# Sets the path to the chrome driver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# directs the browser to visit the root site and sleeps for 2 seconds
browser.visit(root_url)
time.sleep(2)

# Fills out the form for job title and location and sleeps for 2 seconds
browser.find_by_xpath("//input[@id='KeywordSearch']").fill('Data Scientist')
browser.find_by_xpath("//input[@id='LocationSearch']").fill('United States')
browser.find_by_id('HeroSearchButton').click()
time.sleep(2)

# Select date posted drop down and Set date range to search
browser.find_by_text("Date Posted").click()
time.sleep(1)
browser.find_by_text(time_period).click()
time.sleep(2)
  
# initialize page count
n = 1

# run while there is still a next button
while True:
    test_url = browser.url
    meta_message = meta_message
    # finds the script with the page index and links
    try:
        html = browser.find_by_xpath("//script[@type='application/ld+json']")
        meta_dict = eval(BeautifulSoup(html.html,'html.parser').text.replace("\n",""))
        position = meta_dict['itemListElement'][0]['position']
        meta = json.dumps(meta_dict)
    except Exception as e:
        print('ERROR 1\n-------------------------------------------')
        print(e)
    
    #creates directory to save all files
    date = datetime.datetime.strftime(datetime.datetime.now(), '%d-%m-%Y')
    path = corpus_path + date + '/Page_' + str(n) + '/'
    if not os.path.exists(path):
        os.makedirs(path) 

    try:
        with open(path + 'meta_index.json', 'w') as f:
            f.write(meta)
    except Exception as e:
        print('ERROR 2\n-------------------------------------------')
        print(e)
    
    try:
        with open(path + 'meta', 'w') as f:
            f.write(meta_message)
    except Exception as e:
        print('ERROR 3\n-------------------------------------------')
        print(e)
    
    
    
    
    for i in range(30):
        x_button = browser.find_by_css('div.xBtn')
        if x_button:
            x_button.click()
        elements = browser.find_by_xpath("//ul[@class='jlGrid hover']")
        print('##################')
        print(len(elements))
        print('##################')
        element = elements.find_by_css('li.jl')
        
        element[i].click()
        div_box = element.find_by_css('div.flexbox')[i*2]
        blob = str(BeautifulSoup(div_box.html, 'html.parser'))
        print('blob: ' + blob)
            
        try:
            m =re.search('(?<=href=").*(?=" rel)', blob)
            url2 = root_url + m.group(0)
            url2 = url2.replace("amp;","")
            browser.visit(url2)
            time.sleep(2)
        except Exception as e:
            print('ERROR 4\n---------------------------------------------------')
            print(e)
        try:
            html = browser.find_by_xpath("//script[@type='application/ld+json']")
        except Exception as e:
            print('ERROR 4a\n-------------------------------------------')
            print(e)
        try:
            job_dict = eval(BeautifulSoup(html.html,'html.parser').text.replace("\n",""))
            title = job_dict['title']
            company = job_dict['hiringOrganization']['name']
        except Exception as e:
            print('ERROR 4b\n-------------------------------------------')
            print(len(html))
            print(e)
        try:
            # write file to directory
            filename = title + '-' + company + '-pos_' + str(position) + '.json'
            with open(path + filename, 'x') as f:
                f.write(json.dumps(job_dict))
                print('----------------------------------------------------')
                print('File' + filename + 'Written')
                print('----------------------------------------------------')
        except Exception as e:
            print('ERROR 4c\n-------------------------------------------')
            print(e)
            
        if browser.url == test_url:
            pass
        else:
            browser.back()
            time.sleep(2)
        
    
    x_button = browser.find_by_css('div.xBtn')
    if x_button:
        x_button.click()
    next_button = browser.find_by_css('li.next')
    if next_button:
        next_button.click()
        time.sleep(2)
    else:
        break
    n += 1
            
            

##################
1
##################
blob: 
<div>
<a class="jobLink" data-ajax="false" data-ev-a="E-S" data-gd-view="1" data-ja-clk="0" href="/partner/jobListing.htm?pos=101&amp;ao=389188&amp;s=58&amp;guid=000001659111e7d694fcde9d8e142f82&amp;src=GD_JOB_AD&amp;t=SR&amp;extid=1&amp;exst=OL&amp;ist=&amp;ast=OL&amp;vt=w&amp;slr=true&amp;rtp=0&amp;aa=1&amp;ea=1&amp;cs=1_5f42da1a&amp;cb=1535737194908&amp;jobListingId=2889202834" rel="nofollow" target="_blank">Sr. Data Modeler/Architect</a></div>
<div class="alignRt">
<span class="save-job-button save-job-button-2889202834 nowrap " data-ao-id="389188" data-job-id="2889202834" data-save-hook="JOB_SEARCH_PANE" data-saved-job-id="" title="Save">
<i class="heart"></i>
</span>
</div>

ERROR 4c
-------------------------------------------
[Errno 2] No such file or directory: '/Users/grothjd/DS_bootcamp/jobsearch_dashboard/corpus2/31-08-2018/Page_1/Sr. Data Modeler/Architect-Gantec-pos_1.json'
##################
1
##################
blob: <div><s

ElementDoesNotExist: no elements could be found with css "div.flexbox"

In [2]:
n



1

In [4]:
for element in elements:
    print(elements.html)


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578706 (5f725d1b4f0a4acbf5259df887244095596231db),platform=Mac OS X 10.13.6 x86_64)
